## TASK 1

In [2]:
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
appName = "FIFA_project"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder.getOrCreate()

C:\spark\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
spark

In [4]:
df_15 = spark.read.csv("players_15.csv",header=True, inferSchema= True)
df_16 = spark.read.csv("players_16.csv",header=True, inferSchema= True)
df_17 = spark.read.csv("players_17.csv",header=True, inferSchema= True)
df_18 = spark.read.csv("players_18.csv",header=True, inferSchema= True)
df_19 = spark.read.csv("players_19.csv",header=True, inferSchema= True)
df_20 = spark.read.csv("players_20.csv",header=True, inferSchema= True)
df_21 = spark.read.csv("players_21.csv",header=True, inferSchema= True)
df_22 = spark.read.csv("players_22.csv",header=True, inferSchema= True)

In [5]:
columns = df_15.columns
len(columns)

110

In [6]:
#checking if the column names have any special characters
print(df_15.columns == "year")

False


In [7]:
from pyspark.sql.functions import lit 
df_15 = df_15.withColumn("year", lit(2015)) 
df_16 = df_16.withColumn("year", lit(2016)) 
df_17 = df_17.withColumn("year", lit(2017)) 
df_18 = df_18.withColumn("year", lit(2018)) 
df_19 = df_19.withColumn("year", lit(2019)) 
df_20 = df_20.withColumn("year", lit(2020)) 
df_21 = df_21.withColumn("year", lit(2021)) 
df_22 = df_22.withColumn("year", lit(2022)) 

In [8]:
df = df_15.union(df_16.union(df_17.union(df_18.union(df_19.union(df_20.union(df_21.union(df_22)))))))

In [9]:
#checking if sofifa_id is a uniqueID
df.where(df.year == 2020).show(2, vertical = True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | RW, CF, ST           
 overall                     | 94                   
 potential                   | 94                   
 value_eur                   | 9.55E7               
 wage_eur                    | 560000.0             
 age                         | 32                   
 dob                         | 1987-06-24           
 height_cm                   | 170                  
 weight_kg                   | 72                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | RW             

In [10]:
#making a Unique ID
from pyspark.sql.functions import monotonically_increasing_id as mi
id=mi()
df = df.withColumn("Unique_ID", id)

In [11]:
df.show(1, vertical = True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [12]:
db_properties={}
db_properties['username']="postgres"
db_properties['password']="1234shan"
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
db_properties['table']="fifa"
db_properties['driver']="org.postgresql.Driver"

df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

In [13]:
df.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: date (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: date (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = true)
 

### Task 2


In [14]:
#Ingest form postgres
df_fifa = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

#showing sample for postgres table
df_fifa.show(1, vertical=True)
df_fifa.count()
df_fifa.printSchema()

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             


### Question 1:
What are the X clubs that have the highest number of players with contracts 
ending in 2023?
o Use the players that were listed in the 2022 dataset only for answering this 
question.

In [15]:
from pyspark.sql.functions import desc

players22df =  df_fifa.filter(df_fifa['year']==2022)
players22_contract23df =  players22df.filter(players22df['club_contract_valid_until']==2023)
players22_contract23df.groupby('club_name').count().show()


+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|          Göztepe SK|    7|
|        Salford City|    7|
|        FC Voluntari|    7|
|        Guangzhou FC|   11|
|  1. FC Union Berlin|    3|
|          Sagan Tosu|    5|
|  Argentinos Juniors|   10|
|     SC Paderborn 07|    9|
|       Karlsruher SC|    9|
|    Shandong Taishan|    3|
|           Puebla FC|    3|
|     Cheltenham Town|    8|
|San Lorenzo de Al...|   14|
|Universidad Católica|    6|
|           FC Luzern|    6|
|       SC Heerenveen|    9|
|                 AIK|    7|
|  Sandefjord Fotball|    4|
|AC Mineros de Gua...|    4|
|     Rapid București|    7|
+--------------------+-----+
only showing top 20 rows



In [16]:
#displaying in descending order
grouped_counts = players22_contract23df.groupby('club_name').count()
sorted_counts = grouped_counts.orderBy(desc('count'))
sorted_counts.show()

+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|En Avant de Guingamp|   19|
| Club Atlético Lanús|   17|
|       Lechia Gdańsk|   17|
|            Barnsley|   16|
|        Kasimpaşa SK|   16|
|        Bengaluru FC|   16|
|              Al Tai|   15|
|            KAA Gent|   15|
|      Zagłębie Lubin|   15|
|        FC Barcelona|   15|
|          CA Osasuna|   15|
|  SV Wehen Wiesbaden|   15|
|       Coventry City|   15|
|San Lorenzo de Al...|   14|
|          1. FC Köln|   14|
|     Orlando Pirates|   14|
|      Crystal Palace|   14|
|           SV Meppen|   14|
|  Real Valladolid CF|   14|
|         Perth Glory|   14|
+--------------------+-----+
only showing top 20 rows



In [17]:
X = 10  # Replace X with the desired number of top clubs you want to retrieve
top_X_clubs = sorted_counts.limit(X)

top_X_clubs.show()

+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|En Avant de Guingamp|   19|
| Club Atlético Lanús|   17|
|       Lechia Gdańsk|   17|
|            Barnsley|   16|
|        Kasimpaşa SK|   16|
|        Bengaluru FC|   16|
|        FC Barcelona|   15|
|  SV Wehen Wiesbaden|   15|
|          CA Osasuna|   15|
|      Zagłębie Lubin|   15|
+--------------------+-----+



### Question 2:
List the Y clubs with highest average number of players that are older than 27 
years across all years (i.e. count the number of players older than 27 years old
for each club every year, calculate the averages and list the Y clubs with highest 
averages).
o If a club is not available in a specific year, calculate their average on the 
years they are displayed in the dataset. For example, if club A is only 
available in years 2016 and 2017, then count their number of players 
meeting the criteria above and divide the total by 2.
o Make sure to handle this scenario as well: if the user requests 5 clubs with 
highest averages but there are 3 clubs that share the same count at rank 
number 5, please include all of them in your output

In [18]:
from pyspark.sql.functions import col

players_above_27 = df_fifa.filter(col('age') > 27)

In [21]:
from pyspark.sql.window import Window
from pyspark.sql.functions import count, first, col, avg

window_spec = Window.partitionBy('club_name')

club_avg_players = players_above_27.groupBy('club_name', 'year') \
    .agg(count('*').alias('player_count'), first('club_name').over(window_spec).alias('club_name_first')) \
    .groupBy('club_name_first') \
    .agg(avg(col('player_count')).alias('avg_players_count'))

In [22]:
from pyspark.sql.functions import rank

# Calculate rank for each club based on the average number of players
club_avg_players_ranked = club_avg_players.withColumn('rank', rank().over(Window.orderBy(col('avg_players_count').desc())))

# Filter the clubs that share the same rank as the Yth club
Y = 5  # Replace with the desired number of clubs
result = club_avg_players_ranked.filter(col('rank') <= Y+1)

result.show()

+--------------------+-----------------+----+
|     club_name_first|avg_players_count|rank|
+--------------------+-----------------+----+
|                NULL|           109.25|   1|
|  Dorados de Sinaloa|             19.0|   2|
| Matsumoto Yamaga FC|             19.0|   2|
| Shanghai Shenhua FC|             18.5|   4|
|          Qingdao FC|             18.0|   5|
|Club Deportivo Jo...|             17.5|   6|
+--------------------+-----------------+----+



### Question 3:
What is the most frequent nation_position in the dataset for each year? (i.e.
display the most frequent nation_position for 2015, 2016, etc.)

In [27]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window


# Assuming df_fifa is your DataFrame
# Group by 'year' and 'nation_position', count occurrences, and find the most frequent position for each year
most_frequent_nation_positions = df_fifa.groupBy('year', 'nation_position') \
    .agg(count("*").alias("count")) \
    .withColumn("rank", row_number().over(Window.partitionBy("year").orderBy(desc("count")))) \
    .filter(col("rank") == 3) \
    .select("year", "nation_position")


# we do col("rank") == 3 because the first is null then 2nd is sub
most_frequent_nation_positions.show()

+----+---------------+
|year|nation_position|
+----+---------------+
|2015|            LCB|
|2016|            RCB|
|2017|             GK|
|2018|            LCB|
|2019|             GK|
|2020|            RCB|
|2021|            LCB|
|2022|            LCB|
+----+---------------+

